## BeautifulSoup Functions :

In [22]:
import requests
from bs4 import BeautifulSoup

def extract_nav_links(url, class_name, start_pattern):
    # Fetch the HTML content of the webpage
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200 or response.status_code == 412:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the navigation menu by class name
        nav_menu = soup.find(class_=class_name)
        
        # Initialize a list to store the links
        links = []
        
        # Find all anchor tags within the navigation menu
        if nav_menu:
            nav_links = nav_menu.find_all('a')
            
            # Extract the href attribute of each anchor tag
            for link in nav_links:
                if link.has_attr('href') and link['href'].startswith(start_pattern):
                    # Check if the link already exists in the list
                    if link['href'] not in links:
                        links.append(link['href'])
        
        return links
    else:
        # If the request was not successful, print an error message
        print("Failed to fetch the webpage.")
        return []
def scrape_elementor_cta_content(url):
    # Send an HTTP request to the specified URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all 'div' elements with the 'elementor-cta__content' class
        elementor_cta_contents = soup.find_all('div', class_='elementor-cta__content')

        # Extract and return the text content of these elements
        return [div.get_text(strip=True) for div in elementor_cta_contents]
    else:
        # If the page couldn't be fetched, print an error and return None
        print("Failed to fetch the page")
        return None
def scrape_page_content(url):
    # Send an HTTP request to the specified URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Get the main content container, change this if the content is structured differently
        main_content = soup.find('div', class_='elementor-text-editor')  # Adjust the class if needed

        all_text_elements = []

        if main_content:
            # Include tags for headers, paragraphs, unordered lists, and ordered lists
            for element in main_content.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'ul', 'ol']):
                if element.name.startswith('h'):  # Handling headers
                    all_text_elements.append(f"{element.get_text(strip=True)}")
                elif element.name == 'p':  # Handling paragraphs
                    all_text_elements.append(f"{element.get_text(strip=True)}")
                elif element.name in ['ul', 'ol']:  # Handling both unordered and ordered lists
                    list_items = element.find_all('li')
                    list_type = "Ordered list item" if element.name == 'ol' else "Unordered list item"
                    list_content = '\n'.join(f"{list_type}: {li.get_text(strip=True)}" for li in list_items)
                    all_text_elements.append(list_content)
        
        return all_text_elements
    else:
        print(f"Failed to fetch the webpage, status code: {response.status_code}")
        return None
    
def scrape_links_by_attribute(url, attribute_name, visited_urls=None):
    # Initialize a set to store visited URLs if not provided
    if visited_urls is None:
        visited_urls = set()

    # Send an HTTP request to fetch the HTML content
    response = requests.get(url)
    if response.status_code == 200 or response.status_code == 412:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all elements with the specified attribute
        elements = soup.find_all(attrs={attribute_name: True})
        
        # Initialize a list to store scraped data
        scraped_data = []
        
        # Extract the desired data from the current page
        paragraphs = soup.find_all('p')
        paragraph_data = '\n'.join(paragraph.get_text(strip=True) for paragraph in paragraphs)
                        
        tables = soup.find_all('table')
        table_data = []
        for table in tables:
            rows = table.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                row_data = [cell.get_text(strip=True) for cell in cells]
                table_data.append(row_data)

        # Append the scraped data from the current page
        scraped_data.append((url, paragraph_data, table_data))

        # Add the current URL to the set of visited URLs
        visited_urls.add(url)

        # Extract links from the elements
        for element in elements:
            href = element.get(attribute_name)
            if href.startswith('https://fstt.ac.ma/Portail2023') and href not in visited_urls:
                # Recursively scrape the linked page
                data = scrape_links_by_attribute(href, attribute_name, visited_urls)
                scraped_data.extend(data)

        return scraped_data
    else:
        print(f"Failed to fetch URL: {url}")
        return []

## LES FORMATIONS INITIALE :

### LES FORMATIONS DEUST :

In [12]:
# DEUST PAGE URL
url1 = 'https://fstt.ac.ma/Portail2023/deust/'
attribute_name = 'data-wts-url'

# Scrape data using scrape_links_by_class for DEUST URL
data1 = scrape_links_by_attribute(url1, attribute_name)

# Print the scraped data for each URL
print(f"LES FORMATIONS DEUST DE FST TANGER - Lien de page: {url1} \n")
for href, paragraphs, tables in data1:

    # Check if there is content in paragraphs
    if paragraphs.strip():  # This removes any leading/trailing whitespace and checks if the result is non-empty
        print('=' * 100)
        print(f"Lien de page: {href}\n")
        print("Les objectifs de formation:\n")
        print(paragraphs + "\n")

        print("Programme de formation:\n")
        for n in range(len(tables)):
            print(f"Semester {n+1}:")
            for table in tables[n]:
                subjects = [item.strip() for item in table.split("•") if item]
                # Print each subject on a new line
                for subject in subjects:
                    print(f"\t-{subject}")
            print()  # Adds a newline for better separation between semesters

LES FORMATIONS DEUST DE FST TANGER - Lien de page: https://fstt.ac.ma/Portail2023/deust/ 

Lien de page: https://fstt.ac.ma/Portail2023/biologie-chimie-geologie/

Les objectifs de formation:

Le tronc commun BCG est un cursus qui comporte un ensemble de modules pris dans plusieurs champs disciplinaires et ayant pour objectif de faire acquérir des connaissances, des aptitudes et des compétences. Donner à l’étudiant les outils méthodologique pour qu’il puisse organiser son travail et développer un esprit de synthèse, de rigueur et d’initiative.
L’objectif de cette formation et de donner à l’étudiant les possibilités d'orienter sa carrière vers les licences de chimie, de biologie et de géologie.
Également, ce tronc commun permet à l’étudiant de postuler à des concours d’accès aux cycles ingénieurs de notre établissement ainsi qu’au concours nationaux ou internationaux des écoles d’ingénieurs.

Programme de formation:

Semester 1:
	-Biologie cellulaire
	-Optique et Radioactivité
	-Cosmolog

### LES FORMATIONS CYCLE LICENCE :

In [13]:
# URLs obtained from the scrape_links_by_attribute function
url3 = 'https://fstt.ac.ma/Portail2023/cycle-licence/'
attribute_name = 'data-wts-url'

# Scrape data using scrape_links_by_class for  URL
data3 = scrape_links_by_attribute(url3, attribute_name)


# Print the scraped data for each URL
print(f"LES FORMATIONS CYCLE LICENCE DE FST TANGER - Lien de page: {url3} \n")
for href, paragraphs, tables in data3:

    # Check if there is content in paragraphs
    if paragraphs.strip():  # This removes any leading/trailing whitespace and checks if the result is non-empty
        print('=' * 100)
        print(f"Lien de page: {href}\n")
        print("Les objectifs de formation:\n")
        print(paragraphs + "\n")

        print("Programme de formation:\n")
        for n in range(len(tables)):
            print(f"Semester {n+1}:")
            for table in tables[n]:
                subjects = [item.strip() for item in table.split("•") if item]
                # Print each subject on a new line
                for subject in subjects:
                    print(f"\t-{subject}")
            print()  # Adds a newline for better separation between semesters

LES FORMATIONS CYCLE LICENCE DE FST TANGER - Lien de page: https://fstt.ac.ma/Portail2023/cycle-licence/ 

Lien de page: https://fstt.ac.ma/Portail2023/lst-analytique-des-donnees/

Les objectifs de formation:

La Licence Science et Techniques en analytique des données permet aux étudiants de doter de compétences en matière d'outils informatiques, des
techniques et des méthodes statistiques pour permettre d’organiser, de synthétiser et de traduire efficacement les données métier d’une
organisation. L'étudiant doit être en mesure d'apporter un appui analytique à la conduite d'exploration et à l'analyse complexe de données.
• Analyse et fouille de données• Systèmes et réseaux• Ingénierie des données• PFE

Programme de formation:

Semester 1:
	-Mathématiques pour la science des données
	-Structures des données avancées et théorie des graphes
	-Fondamentaux des bases de données
	-Algorithmique avancée et programmation
	-Développement WEB
	-Développement personnel et intelligence émotionnell

### LES FORMATIONS CYCLE MASTRE :

In [14]:
# URLs obtained from the scrape_links_by_attribute function
url4 = 'https://fstt.ac.ma/Portail2023/cycle-master/'
attribute_name = 'data-wts-url'

# Scrape data using scrape_links_by_class for URL
data4 = scrape_links_by_attribute(url4, attribute_name)

# Print the scraped data for each URL
print(f"LES FORMATIONS CYCLE MASTER - Lien de page: {url4} \n")
for href, paragraphs, tables in data4:

    # Check if there is content in paragraphs
    if paragraphs.strip():  # This removes any leading/trailing whitespace and checks if the result is non-empty
        print('=' * 100)
        print(f"Lien de page: {href}\n")
        print("Les objectifs de formation:\n")
        print(paragraphs + "\n")

        print("Programme de formation:\n")
        for n in range(len(tables)):
            print(f"Semester {n+1}:")
            for table in tables[n]:
                subjects = [item.strip() for item in table.split("•") if item]
                # Print each subject on a new line
                for subject in subjects:
                    print(f"\t-{subject}")
            print()  # Adds a newline for better separation between semesters

LES FORMATIONS CYCLE MASTER - Lien de page: https://fstt.ac.ma/Portail2023/cycle-master/ 

Lien de page: https://fstt.ac.ma/Portail2023/mst-analyse-appliquee-etingenierie-statistique/

Les objectifs de formation:

Le Master Analyse Appliquée et Ingénierie Statistique (MAAIS) de l'Université Abdelmalek Essaadi a pour vocation d'offrir des parcours qui trouvent leurs bases dans une approche pluridisciplinaire. Au département de mathématiques on propose aux étudiants d’acquérir une double compétence en mathématiques et en statistiques appliquées s’appuyant sur un tronc commun développant des concepts et des outils de base. Pour une approche pluridisciplinaire des problèmes liés aux applications, les mathématiques et les statistiques appliquées ne peuvent que former des étudiants pour la recherche et les activités professionnelles de haut niveau avec un bon bagage mathématique. La spécialisation se ferait au terme du second semestre (S2) selon le choix des étudiants. En plus une bonne form

## LES CLUBS DE FSTT:

In [15]:
# Function to check and possibly replace the URL
def check_and_replace_url(url):
    if url == "https://fstt.ac.ma/Portail2023/di-geoinformation/":
        return "https://fstt.ac.ma/Portail2023/club-geoinformation/"
    return url

# URLs obtained from the scrape_links_by_attribute function
url5 = 'https://fstt.ac.ma/Portail2023/clubs/'
attribute_name = 'data-wts-url'
data5 = scrape_links_by_attribute(url5, attribute_name)

# Print the scraped data for each URL
print(f"LES CLUBS DE FST TANGER - Lien de page: {url5} \n")
for href, paragraphs, tables in data5:

    # Check if the URL needs to be replaced
    modified_href = check_and_replace_url(href)

    # Scrape data from the modified URL if different
    if modified_href != href:
        modified_data = scrape_links_by_attribute(modified_href, attribute_name)
        for m_href, m_paragraphs, m_tables in modified_data:
            if m_paragraphs.strip():
                print('=' * 100)
                print(f"Lien de page: {m_href}\n")
                print("Les objectifs de formation:\n")
                print(m_paragraphs + "\n")
            
    else:
        if paragraphs.strip():  # This removes any leading/trailing whitespace and checks if the result is non-empty
            print('=' * 100)
            print(f"Lien de page: {href}\n")
            print("Les objectifs de formation:\n")
            print(paragraphs + "\n")

LES CLUBS DE FST TANGER - Lien de page: https://fstt.ac.ma/Portail2023/clubs/ 

Lien de page: https://fstt.ac.ma/Portail2023/greenology/

Les objectifs de formation:

Le Club de l’environnement de la FST de Tanger, Dénommé Greenology Club, est nouvellement créé et il a comme vocation de : – Transmettre aux étudiants, une conscience nationale éclairée, un patriotisme activé pour un environnement préservé. – Sensibiliser et rendre les étudiants de la FST de Tanger des combattants contre l’éparpillement de l’environnement. – Tous unis, bien convaincus que nos richesses environnementales très précieuses sont menacées, et qu’avec une conscience large du désastre écologique qui frappe à la porte de notre pays, on pourra gagner la bataille et sauver ce qu’il reste encore à sauver.
la page facebook:
https://www.facebook.com/greenology.club
Email de contact :
Responsable d’activités4saaddine@gmail.com

Lien de page: https://fstt.ac.ma/Portail2023/club-d-art-dramatique-et-d-animation-culturelle/

Lien de page: https://fstt.ac.ma/Portail2023/club-geoinformation/

Les objectifs de formation:

Le Club Géoinformation est créé par les élèves ingénieurs de la filière Géoinformation (plusque 80 membres), fondé en 2009.Ce Club a pour tâche l’organisation des conférences, projections des documentaires géomatiques, réalisation des panneaux d’affichage, constructions des bases de données des cours magistraux, organisation de sorties pédagogique et des visites d’entreprises et organisations des tables rondes pour la discussion des dernières nouveautés de la géomatique et la présentation de la FSTT dans des séminaires et colloques national et international .
Pour une bonne organisation et une bonne gestion le club a quatre cellules à savoir :
·Cellule communicationqui a comme taches principales :
?             Aider à établir des échanges d’information claire, constante et pertinente aux membres du Club sur une base régulière et continue.
?             Informer les médias de se présentés da

## LES DEPARTEMENTS DE FST TANGER :

In [19]:
# URL to scrape
url6 = "https://fstt.ac.ma/Portail2023/les-departement/"

# Scrape the page
elementor_cta_contents = scrape_elementor_cta_content(url6)
if elementor_cta_contents:
    print("LES DEPARTEMENTS DE FST TANGER :\n")
    for content in elementor_cta_contents:
        print(content)

LES DEPARTEMENTS DE FST TANGER :

GÉNIE INFORMATIQUEChef : Pr.EL BRAK MohamedEmail :melbrak@uae.ac.ma
GÉNIE CHIMIQUEChef : Pr.CHABBI MohamedEmail :mchabbi@uae.ac.ma
SCIENCES DE LA TERREChef : Pr.BOULAASSAL HakimEmail :h.boulaassal@uae.ac.ma
GÉNIE MÉCANIQUEChef : Pr.ELAYACHI IlhamEmail :i.elayachi@uae.ac.ma
SCIENCES DE LA VIEChef : Pr.HASSANI ZERROUK MounirEmail: mhassani@uae.ac.ma
GÉNIE ELECTRIQUEChef : Pr.HADJ BARAKA IbrahimEmail :ihadjbaraka@uae.ac.ma
TECChef : Pr.OUCHEN MOHAMEDEmail :mouchen@uae.ac.ma
PHYSIQUEChef  : Pr.AJDOUR MouniaEmail :majdour@uae.ac.ma
MATHÉMATIQUESChef : Pr.EL HALIMI RACHIDEmail :r.elhalimi@uae.ac.ma


## PRESENTATION DE LA FST DE TANGER & DIAGNOSIQUE :

In [23]:
# URL to scrape
url7 = "https://fstt.ac.ma/Portail2023/presentation/"

# Scrape the page
content = scrape_page_content(url7)
if content:
    for text in content:
        print(text)

I- PRESENTATION DE LA FST DE TANGER & DIAGNOSIQUE
La Faculté des Sciences et Techniques de Tanger (FSTT), relevant de l’Université Abdelmalek Essaâdi a été créée en 1995. Elle fait partie des établissements de l’enseignement supérieur à accès régulé et a pour missions la formation initiale dans les domaines scientifiques et techniques, la formation continue ainsi que la recherche et le développement dans les domaines des sciences et techniques.

Formation Initiale
La FST de Tanger offre des cursus de formation qui préparent aux diplômes suivants :
Unordered list item: DEUST : Diplôme d’Etudes Universitaire en Sciences et Techniques (Bac +2).
Unordered list item: LST : Diplôme de Licence en Sciences et Techniques (Bac +3)
Unordered list item: MST : Diplôme de Master en Sciences et Techniques (Bac +5)
Unordered list item: DI : Diplôme d’Ingénieur d’État (Bac +5)
Unordered list item: Doctorat en Sciences et Techniques (Bac +8)
La FST de Tanger propose une panoplie de formation riches et d

## LE MOT DU DOYEN :

In [25]:
url8 = "https://fstt.ac.ma/Portail2023/mot-du-doyen/"

# Scrape the page
content = scrape_page_content(url8)
if content:
    print("LE MOT DU DOYEN :")
    for text in content:
        print(text)

LE MOT DU DOYEN :

Le monde d’aujourd’hui connait des évolutions majeures et complexes caractérisées par la globalisation des échanges et l’intensification de la concurrence. Le Maroc n’échappe pas à cette tendance. Il est confronté à des enjeux de développement et de croissance basés essentiellement sur le capital humain. D’où la nécessité, pour le Maroc, de se doter d’un système d’éducation et de formation performant productif de savoir et innovateur afin de produire des compétences et des profils qualifiés, une jeunesse en mesure de répondre au besoin du marché et de prendre en charge tant les entreprises des divers secteurs de l’économie que les organismes investis de missions de service public, de même que les chercheurs et les intellectuels indispensables au développement d’une société du savoir et d’économie.
Notre projet de développement de la FST de Tanger est conscient du rôle que doit jouer cette dernière dans le développement socio-économique régional et national où elle ne

## Page Conseil de l’établissement :

In [39]:
url9 = 'https://fstt.ac.ma/Portail2023/conseil-de-letablissement/'
data9 = scrape_links_by_attribute(url9, attribute_name)
print(f"LA PAGE DE Conseil de l’établissement - Lien de page: {url9}\n")
for idx, (href, paragraphs, tables) in enumerate(data9, start=1):
    print(f"{paragraphs}")
    for entry in tables:
        if entry:  # This checks if the list is not empty
            name, responsibility = entry
            print(f"-Member : {name} - Responsabilité : {responsibility}")

LA PAGE DE Conseil de l’établissement - Lien de page: https://fstt.ac.ma/Portail2023/conseil-de-letablissement/

La Faculté est gérée par un conseil d’établissement présidé par le Doyen. Le conseil de l’établissement comprend des membres de droit, des représentants élus des personnels enseignants et des personnels administratifs et techniques, des représentants élus des étudiants, ainsi que des membres désignés parmi des personnalités extérieures.
-Member : DIANI Mustapha - Responsabilité : Doyen
-Member : JBILOU Mohammed - Responsabilité : Vice Doyen de la Formation
-Member : OUARDOUZ Mustapha - Responsabilité : Vice Doyen de la Recherche et de la coopération
-Member : BIHRI Hassan - Responsabilité : SG chargé du secrétariat du conseil
-Member : AJDOUR Mounia - Responsabilité : Chef du département de Physique
-Member : CHABBI Mohamed - Responsabilité : Chef du département de Génie Chimique
-Member : EL HALIMI Rachid - Responsabilité : Chef du département des Mathématiques
-Member : EL

## Page de Commission Scientifique :

In [43]:
url10 = 'https://fstt.ac.ma/Portail2023/commission-scientifique/'
data10 = scrape_links_by_attribute(url10, attribute_name)

print(f"LA PAGE DE Commission Scientifique - Lien de page: {url10}\n")
print("Membres de la Commission Scientifique émanant du conseil de l’établissement:")
for idx, (href, paragraphs, tables) in enumerate(data10, start=1):
    print(f"{paragraphs}")
    for entry in tables:
        if entry:  # This checks if the list is not empty
            name, responsibility = entry
            print(f"-Member : {name} - Responsabilité : {responsibility}")

LA PAGE DE Commission Scientifique - Lien de page: https://fstt.ac.ma/Portail2023/commission-scientifique/

Membres de la Commission Scientifique émanant du conseil de l’établissement:

-Member : DIANI Mustapha - Responsabilité : Doyen
-Member : OUARDOUZ Mustapha - Responsabilité : Vice Doyen de la Recherche et de la coopération
-Member : JBILOU Mohammed - Responsabilité : Vice Doyen de la Formation
-Member : ARHRIB Abdesslam - Responsabilité : P.E.S
-Member : EL HARZLI Mohamed - Responsabilité : P.E.S
-Member : EN-NAIMI El Mokhtar - Responsabilité : P.E.S
-Member : ROSSI Abdelhamid - Responsabilité : P.E.S
-Member : AMHARREF Mina - Responsabilité : P.E.S (Désignée)
-Member : CHAFIK Tarik - Responsabilité : P.E.S (Désigné)
-Member : AJDOUR Mounia - Responsabilité : Chef du département de Physique
-Member : CHABBI Mohamed - Responsabilité : Chef du département de Génie Chimique
-Member : EL HALIMI Rachid - Responsabilité : Chef du département des Mathématiques
-Member : EL BRAK Mohamed 

## Page de Commission de la Recherche Scientifique et de Coopération:

In [50]:
url11 = 'https://fstt.ac.ma/Portail2023/commission-recherche-scientifique-cooperation/'
data11 = scrape_links_by_attribute(url11, attribute_name)

print(f"LA PAGE DE Commission Scientifique-Lien de page: {url11}\n")
print("Commission de la Recherche Scientifique et de Coopération émanant du conseil de l’établissement")
for href, paragraphs, tables in data11:
    print(f"{paragraphs}")
    for entry in tables:
        if entry:  # This checks if the list is not empty
            name, responsibility = entry
            print(f"-Member : {name} - Responsabilité : {responsibility}")

LA PAGE DE Commission Scientifique-Lien de page: https://fstt.ac.ma/Portail2023/commission-recherche-scientifique-cooperation/

Commission de la Recherche Scientifique et de Coopération émanant du conseil de l’établissement

-Member : DIANI Mustapha - Responsabilité : Doyen
-Member : OUARDOUZ Mustapha - Responsabilité : Vice Doyen de la Recherche et de la coopération
-Member : JBILOU Mohammed - Responsabilité : Vice Doyen de la Formation
-Member : AJDOUR Mounia - Responsabilité : Chef du département de Physique
-Member : CHABBI Mohamed - Responsabilité : Chef du département de Génie Chimique
-Member : EL HALIMI Rachid - Responsabilité : Chef du département des Mathématiques
-Member : EL BRAK Mohamed - Responsabilité : Chef du département de Génie Informatique
-Member : BOULAASSAL Hakim - Responsabilité : Chef du département des Sciences de la Terre
-Member : HADJ BARAKA Ibrahim - Responsabilité : Chef du département de Génie Électrique
-Member : ELAYACHI Ilham - Responsabilité : Chef d

## Page de Commission Pédagogique:

In [49]:
url12 = 'https://fstt.ac.ma/Portail2023/commission-pedagogique/'
data12 = scrape_links_by_attribute(url12, attribute_name)

print(f"LA PAGE DE Commission Pédagogique - Lien de page: {url12}\n")
print("Membres de la commission pédagogique émanant du conseil de l’établissement")
for href, paragraphs, tables in data12:
    print(f"{paragraphs}")
    for entry in tables:
        if entry:  # This checks if the list is not empty
            name, responsibility = entry
            print(f"-Member : {name} - Responsabilité : {responsibility}")

LA PAGE DE Commission Pédagogique - Lien de page: https://fstt.ac.ma/Portail2023/commission-pedagogique/

Membres de la commission pédagogique émanant du conseil de l’établissement

-Member : DIANI Mustapha - Responsabilité : Doyen
-Member : JBILOU Mohammed - Responsabilité : Vice Doyen de la Formation
-Member : OUARDOUZ Mustapha - Responsabilité : Vice Doyen de la Recherche et de la coopération
-Member : AJDOUR Mounia - Responsabilité : Chef du département de Physique
-Member : CHABBI Mohamed - Responsabilité : Chef du département de Génie Chimique
-Member : EL HALIMI Rachid - Responsabilité : Chef du département des Mathématiques
-Member : BOULAASSAL Hakim - Responsabilité : Chef du département des Sciences de la Terre
-Member : HADJ BARAKA Ibrahim - Responsabilité : Chef du département de Génie Électrique
-Member : ELAYACHI Ilham - Responsabilité : Chef du département de Génie Mécanique
-Member : OUCHEN Mohamed - Responsabilité : Chef du département de Langue et Communication
-Membe

## Page Commission d’Activités Culturelles et Sportives:

In [51]:
url13 = 'https://fstt.ac.ma/Portail2023/commission-activites-culturelles-et-sportives/'
data13 = scrape_links_by_attribute(url13, attribute_name)

print(f"LA PAGE DE Commission d’Activités Culturelles et Sportives \n-Lien de page: {url13}\n")
print("Commission d'Activités Culturelles et Sportives émanant du conseil de l’établissement")
for href, paragraphs, tables in data13:
    print(f"{paragraphs}")
    for entry in tables:
        if entry:  # This checks if the list is not empty
            name, responsibility = entry
            print(f"-Member : {name} - Responsabilité : {responsibility}")

LA PAGE DE Commission d’Activités Culturelles et Sportives 
-Lien de page: https://fstt.ac.ma/Portail2023/commission-activites-culturelles-et-sportives/

Commission d'Activités Culturelles et Sportives émanant du conseil de l’établissement

-Member : DIANI Mustapha - Responsabilité : Doyen
-Member : JBILOU Mohammed - Responsabilité : Vice Doyen de la Formation
-Member : OUARDOUZ Mustapha - Responsabilité : Vice Doyen de la Recherche et de la coopération
-Member : AJDOUR Mounia - Responsabilité : Chef du département de Physique
-Member : CHABBI Mohamed - Responsabilité : Chef du département de Génie Chimique
-Member : EL HALIMI Rachid - Responsabilité : Chef du département des Mathématiques
-Member : EL BRAK Mohamed - Responsabilité : Chef du département de Génie Informatique
-Member : BOULAASSAL Hakim - Responsabilité : Chef du département des Sciences de la Terre
-Member : HADJ BARAKA Ibrahim - Responsabilité : Chef du département de Génie Électrique
-Member : ELAYACHI Ilham - Respons

## Page Commission de suivi du budget :

In [54]:
url14 = 'https://fstt.ac.ma/Portail2023/commission-suivi-du-budget/'
data14 = scrape_links_by_attribute(url14, attribute_name)

print(f"LA PAGE DE Commission de suivi du budget \n-Lien de page: {url14}\n")
print("Membres de la commission de suivi du budget émanant du conseil de l’établissement")
for href, paragraphs, tables in data14:
    print(f"{paragraphs}")
    for entry in tables:
        if entry:  # This checks if the list is not empty
            name, responsibility = entry
            print(f"-Member : {name} - Responsabilité : {responsibility}")

LA PAGE DE Commission de suivi du budget 
-Lien de page: https://fstt.ac.ma/Portail2023/commission-suivi-du-budget/

Membres de la commission de suivi du budget émanant du conseil de l’établissement

-Member : DIANI Mustapha - Responsabilité : Doyen
-Member : JBILOU Mohammed - Responsabilité : Vice Doyen de la Formation
-Member : OUARDOUZ Mustapha - Responsabilité : Vice Doyen de la Recherche et de la coopération
-Member : AJDOUR Mounia - Responsabilité : Chef du département de Physique
-Member : CHABBI Mohamed - Responsabilité : Chef du département de Génie Chimique
-Member : EL HALIMI Rachid - Responsabilité : Chef du département des Mathématiques
-Member : BOULAASSAL Hakim - Responsabilité : Chef du département des Sciences de la Terre
-Member : HADJ BARAKA Ibrahim - Responsabilité : Chef du département de Génie Électrique
-Member : ELAYACHI Ilham - Responsabilité : Chef du département de Génie Mécanique
-Member : OUCHEN Mohamed - Responsabilité : Chef du département de Langue et Co